In [28]:
import torch
from torch import nn
torch.set_printoptions(profile="full")

In [82]:
x = torch.rand((1,3,224,224))
print(x.shape)

# patch embed
print('patch embed')
patch_size=4
in_c=3
embed_dim=96
proj = nn.Conv2d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)

_, _, H, W = x.shape
print(x.shape)
x = proj(x)
_, _, H, W = x.shape
print(x.shape)
x = x.flatten(2).transpose(1,2)
print(x.shape)
print('H,W:',H,W)

print('='*50)
print('basic layer [stage2 swinT block and stage3 patch merging]')  # 96, 192
H,W = 28,28
print('H,W:',H,W)
x = torch.rand(2,28*28,192)
print(x.shape)
print('='*30)
print('mask')
img_mask = torch.zeros((1,H,W,1))
h_slices = (slice(0,-7),slice(-7,-3),slice(-3,None))
w_slices = (slice(0,-7),slice(-7,-3),slice(-3,None))
cnt = 0
for h in h_slices:
    for w in w_slices:
        img_mask[:,h,w,:] = cnt
        cnt += 1
print(img_mask[0,-7:-3,-7:-3,0])
print(img_mask.shape)

windows = img_mask.view(1,H//7,7,W//7,7,1)
print(windows.shape)

mask_windows = windows.permute(0,1,3,2,4,5).contiguous().view(-1,7,7,1)
print(mask_windows.shape)
print(mask_windows[-1,:,:,0])

mask_windows = mask_windows.view(-1,7*7)
print(mask_windows.shape)

attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
print(attn_mask.shape)
# print(attn_mask[-1,:])

attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
print('attn mask shape',attn_mask.shape)
print('='*30)
print('swinT block')
print('x shape',x.shape)
shortcut = x 
x = x.view(2,H,W,192)
attn_mask_none = None
shifted_x = x
window_size = 7
shifted_x = shifted_x.view(2,H//7,7,W//7,7,192)
x_windows = shifted_x.permute(0,1,3,2,4,5).contiguous().view(-1,7,7,192)
print('x_windows shape',x_windows.shape)
x_windows = x_windows.view(-1,7*7,192)
print('x_windows shape',x_windows.shape)
B_,N,C = x_windows.shape
qkv = nn.Linear(192, 192 * 3, bias=False)
x_window_qkv = qkv(x_windows)
print('x_window qkv shape',x_window_qkv.shape)
num_heads  = 6
x_window_qkv = x_window_qkv.reshape(B_,N,3,num_heads,192//num_heads)
print('x_window qkv shape',x_window_qkv.shape)
x_window_qkv = x_window_qkv.permute(2,0,3,1,4)
print('x_window qkv shape',x_window_qkv.shape)
q,k,v = x_window_qkv.unbind(0)
print('q k v shape',q.shape)
attn = q @ k.transpose(-2,-1)
print('attn shape',attn.shape)
attn_window = attn @ v
print('attn window shape',attn_window.shape)
attn_window = attn_window.transpose(1,2).reshape(B_,N,C)
print('attn window shape',attn_window.shape)
attn_window = attn_window.view(-1,7,7,192)
print('attn window shape',attn_window.shape)
B = int(attn_window.shape[0] / (H*W/7/7))
shift_x =  attn_window.view(B,H//7,W//7,7,7,-1)
print('shift x shape',shift_x.shape)
shift_x = shift_x.permute(0,1,3,2,4,5).contiguous()
print('shift x shape',shift_x.shape)
shift_x = shift_x.view(B,H,W,-1)
print('shift x shape',shift_x.shape)
x = shift_x.view(B,H*W,C)
print('x shape',x.shape)

print('='*30)
print('shift swinT block')
print('x shape',x.shape)
x = x.view(B,H,W,C) 
print('x view shape',x.shape)
shift_x = torch.roll(x,shifts=(-3,-3),dims=(1,2))
print('shift x shape',shift_x.shape)
B,H,W,C = shift_x.shape
x_windows = shift_x.view(B,H//7,7,W//7,7,C)
print('x window shape',x_windows.shape)
x_windows = x_windows.permute(0,1,3,2,4,5).contiguous().view(-1,7,7,C)
print('x window shape',x_windows.shape)
x_windows = x_windows.view(-1,7*7,C)
print('x window shape',x_windows.shape)
B_, N, C = x_windows.shape
qkv = nn.Linear(192, 192 * 3, bias=False)
x_window_qkv = qkv(x_windows)
print('x_window qkv shape',x_window_qkv.shape)
num_heads  = 6
x_window_qkv = x_window_qkv.reshape(B_,N,3,num_heads,192//num_heads)
print('x_window qkv shape',x_window_qkv.shape)
x_window_qkv = x_window_qkv.permute(2,0,3,1,4)
print('x_window qkv shape',x_window_qkv.shape)
q,k,v = x_window_qkv.unbind(0)
print('q k v shape',q.shape)
attn = q @ k.transpose(-2, -1)
print('attn shape',attn.shape)
print('attn mask shape',attn_mask.shape)
nw = attn_mask.shape[0]
attn = attn.view(B_//nw,nw,num_heads,N,N)
print('attn view shape',attn.shape)
attn_mask = attn_mask.unsqueeze(1).unsqueeze(0)
print('attn mask unsqueeze shape',attn_mask.shape)
attn = attn + attn_mask
print('attn in mask shape',attn.shape)
attn = attn.view(-1,num_heads,N,N)
print('attn view shape',attn.shape)
softmax = nn.Softmax(dim=-1)
attn = softmax(attn)
attn_windows = (attn @ v)
print('attn window shape',attn_windows.shape)
attn_windows = attn_windows.transpose(1,2).reshape(B_,N,C)
print('attn windows shape',attn_windows.shape)
attn_windows = attn_windows.view(-1,7,7,C)
print('attn windows shape',attn_windows.shape)
B = int(attn_windows.shape[0]/H/W*7*7)
shift_x = attn_windows.view(B,H//7,W//7,7,7,C)
print('shift x shape',shift_x.shape)
shift_x = shift_x.permute(0,1,3,2,4,5)
print('shift x shape',shift_x.shape)
shift_x = shift_x.contiguous().view(B,H,W,-1)
print('shift x shape',shift_x.shape)
x = torch.roll(shift_x,shifts=(3,3),dims=(1,2))
print('roll back x shape',x.shape)
x = x.view(B,H*W,C)
print('x view shape',x.shape)
print('='*30)
print('downsample')
B,L,C = x.shape
print('x shape',x.shape)
x = x.view(B,H,W,C)
print('x view shape',x.shape)
x0 = x[:, 0::2, 0::2, :]  # [B, H/2, W/2, C]
x1 = x[:, 1::2, 0::2, :]  # [B, H/2, W/2, C]
x2 = x[:, 0::2, 1::2, :]  # [B, H/2, W/2, C]
x3 = x[:, 1::2, 1::2, :]  # [B, H/2, W/2, C]
x = torch.cat([x0,x1,x2,x3],-1)
print('x cat shape',x.shape)
x = x.view(B,-1,4*C)
print('x view shape',x.shape)
reduction = nn.Linear(4 * 192, 2 * 192, bias=False)
x = reduction(x)
print('x reduction shape',x.shape)

torch.Size([1, 3, 224, 224])
patch embed
torch.Size([1, 3, 224, 224])
torch.Size([1, 96, 56, 56])
torch.Size([1, 3136, 96])
H,W: 56 56
basic layer [stage2 swinT block and stage3 patch merging]
H,W: 28 28
torch.Size([2, 784, 192])
mask
tensor([[4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.]])
torch.Size([1, 28, 28, 1])
torch.Size([1, 4, 7, 4, 7, 1])
torch.Size([16, 7, 7, 1])
tensor([[4., 4., 4., 4., 5., 5., 5.],
        [4., 4., 4., 4., 5., 5., 5.],
        [4., 4., 4., 4., 5., 5., 5.],
        [4., 4., 4., 4., 5., 5., 5.],
        [7., 7., 7., 7., 8., 8., 8.],
        [7., 7., 7., 7., 8., 8., 8.],
        [7., 7., 7., 7., 8., 8., 8.]])
torch.Size([16, 49])
torch.Size([16, 49, 49])
attn mask shape torch.Size([16, 49, 49])
swinT block
x shape torch.Size([2, 784, 192])
x_windows shape torch.Size([32, 7, 7, 192])
x_windows shape torch.Size([32, 49, 192])
x_window qkv shape torch.Size([32, 49, 576])
x_window qkv shape torch.Size([32, 49, 3, 6, 

In [59]:
x = torch.arange(1,82).reshape(9,9)
print(x)
shift_x = torch.roll(x,shifts=(-1,-1),dims=(0,1))
print(shift_x)

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24, 25, 26, 27],
        [28, 29, 30, 31, 32, 33, 34, 35, 36],
        [37, 38, 39, 40, 41, 42, 43, 44, 45],
        [46, 47, 48, 49, 50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59, 60, 61, 62, 63],
        [64, 65, 66, 67, 68, 69, 70, 71, 72],
        [73, 74, 75, 76, 77, 78, 79, 80, 81]])
tensor([[11, 12, 13, 14, 15, 16, 17, 18, 10],
        [20, 21, 22, 23, 24, 25, 26, 27, 19],
        [29, 30, 31, 32, 33, 34, 35, 36, 28],
        [38, 39, 40, 41, 42, 43, 44, 45, 37],
        [47, 48, 49, 50, 51, 52, 53, 54, 46],
        [56, 57, 58, 59, 60, 61, 62, 63, 55],
        [65, 66, 67, 68, 69, 70, 71, 72, 64],
        [74, 75, 76, 77, 78, 79, 80, 81, 73],
        [ 2,  3,  4,  5,  6,  7,  8,  9,  1]])
